# Generate queries with neurocard

See neurocard.ipynb in https://github.com/Erostrate9/neurocard

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp /content/drive/MyDrive/job-light-content-2500.csv /content/job-light-content-2500.csv

- Notice: As mentioned in Fauce paper, JOB-base contains the queries in JOB-base generated based on numeric columns in JOB-light.
- So instead of any String column, there're only int and float columns in the training data.
- There're only 6 tables in JOB-base:
  1. title
  2. cast_info
  3. movie_info
  4. movie_company
  5. movie_keyword
  6. movie_info_idx


# Queries Featurization

## Clone GitHithub repository

In [1]:
!git config --global user.email ericsun42@outlook.com
!git config --global user.name Erostrate9
!mkdir -p /root/.ssh && cp -r "/content/drive/My Drive/ssh/." /root/.ssh/
!ssh -T git@github.com


Hi Erostrate9! You've successfully authenticated, but GitHub does not provide shell access.


In [2]:
!git clone -b eric git@github.com:Erostrate9/vldb2021_fauce.git

Cloning into 'vldb2021_fauce'...
remote: Enumerating objects: 787, done.
remote: Counting objects: 100% (345/345), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 787 (delta 133), reused 284 (delta 85), pack-reused 442 (from 1)
Receiving objects: 100% (787/787), 4.80 MiB | 13.39 MiB/s, done.
Resolving deltas: 100% (353/353), done.


In [ ]:
!mkdir py3
!cd py3 && git clone -b dverma/update_fauce git@github.com:Erostrate9/vldb2021_fauce.git

Cloning into 'vldb2021_fauce'...
remote: Enumerating objects: 757, done.
remote: Counting objects: 100% (315/315), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 757 (delta 117), reused 269 (delta 84), pack-reused 442 (from 1)
Receiving objects: 100% (757/757), 4.75 MiB | 13.52 MiB/s, done.
Resolving deltas: 100% (337/337), done.


In [101]:
# commit update && push
%%bash
cd vldb2021_fauce
git add .
git commit -m "update Columns Encoding embedding"
git push origin eric

[eric 461589f] update Columns Encoding embedding
 4 files changed, 24 insertions(+), 26 deletions(-)
 create mode 100644 queries_featurization/graph_embedding/emb/global_columns.emd
 delete mode 100644 queries_featurization/graph_embedding/graph/global_column_schema.edgelist
 create mode 100644 queries_featurization/graph_embedding/graph/global_columns_dependency_graph.edgelist


To github.com:Erostrate9/vldb2021_fauce.git
   9fce123..461589f  eric -> eric


# Install environment

In [4]:
!apt install build-essential
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda install --channel defaults conda --yes
!conda update --channel defaults --all --yes

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
--2024-08-20 20:26:43--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.191.158, 104.16.32.241, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.191.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 146836934 (140M) [application/octet-stream]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>] 140.03M   502MB/s    in 0.3s    

2024-08-20 20:26:43 (502 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [146836934/146836934]

PREFIX=/usr/local
Unpacking payload ...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
instal

In [ ]:
!conda env remove -n qf

In [5]:
%%bash
conda env create -f /content/vldb2021_fauce/queries_featurization/environment.yml
curl https://bootstrap.pypa.io/pip/2.7/get-pip.py -o get-pip.py
source activate qf
python get-pip.py --force-reinstall
pip install networkx==1.11
pip install numpy==1.11.2
pip install gensim==0.12.1
pip install tensorflow==0.12.1
pip install joblib==0.11
pip install scikit-learn
pip install singledispatch

Channels:
 - conda-forge
 - defaults
 - prometeia
 - legel
 - free
Platform: linux-64
Solving environment: ...working... done

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
#
# To activate this environment, use
#
#     $ conda activate qf
#
# To deactivate an active environment, use
#
#     $ conda deactivate

  Attempting uninstall: networkx
    Found existing installation: networkx 2.2
    Uninstalling networkx-2.2:
      Successfully uninstalled networkx-2.2
  Attempting uninstall: joblib
    Found existing installation: joblib 0.14.1
    Uninstalling joblib-0.14.1:
      Successfully uninstalled joblib-0.14.1


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1863k  100 1863k    0     0  10.0M      0 --:--:-- --:--:-- --:--:-- 10.0M


In [ ]:
!pip install -r /content/py3/vldb2021_fauce/requirements.txt

## Table Encoding using a graph embedding method

- The whole schema of IMDB database is shown in the following figure.
However, there're only 6 tables in JOB-base:
  1. title
  2. cast_info
  3. movie_info
  4. movie_company
  5. movie_keyword
  6. movie_info_idx
![IMDB schema](https://www.researchgate.net/profile/Peter-Boncz/publication/319893076/figure/fig2/AS:631637725438007@1527605577677/MDB-schema-with-key-foreign-key-relationships-Underlined-attributes-are-primary-keys.png)

- Fauce paper didn't record full details of training data. They didn't mention which 15 tables they used, and the edgelist they offered doesn't correspond to the IMDB schema.
- So I use the JOB-base version with 6 tables.
```csv
title 1
cast_info 2
movie_info 3
movie_companies 4
movie_keyword 5
movie_info_idx 6
```



In [8]:
%%bash
conda env create -f /content/vldb2021_fauce/queries_featurization/environment.yml
curl https://bootstrap.pypa.io/pip/2.7/get-pip.py -o get-pip.py
source activate qf
python get-pip.py --force-reinstall
pip install networkx==1.11
pip install numpy==1.11.2
pip install gensim==0.12.1
pip install tensorflow==0.12.1
pip install joblib==0.11
pip install scikit-learn
pip install singledispatch

Channels:
 - conda-forge
 - defaults
 - prometeia
 - legel
 - free
Platform: linux-64
Solving environment: ...working... done

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
#
# To activate this environment, use
#
#     $ conda activate qf
#
# To deactivate an active environment, use
#
#     $ conda deactivate

  Attempting uninstall: networkx
    Found existing installation: networkx 2.2
    Uninstalling networkx-2.2:
      Successfully uninstalled networkx-2.2
  Attempting uninstall: joblib
    Found existing installation: joblib 0.14.1
    Uninstalling joblib-0.14.1:
      Successfully uninstalled joblib-0.14.1


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1863k  100 1863k    0     0  7991k      0 --:--:-- --:--:-- --:--:-- 7997k


In [ ]:
# m =6, dimensions for each table = [log(m+1)] = 3
%%bash
cd /content/vldb2021_fauce/queries_featurization/graph_embedding
source activate qf
python main.py --input graph/graph.edgelist --output emb/graph_node.emd --dimensions 3

Walk iteration:
1 / 3
2 / 3
3 / 3


/usr/local/envs/qf/lib/python2.7/site-packages/gensim/models/word2vec.py:651: UserWarning: C extension not loaded for Word2Vec, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn("C extension not loaded for Word2Vec, training will be slow. "


The first line has the following format:

`num_of_nodes dim_of_representation`

The next lines are as follows:

`node_id dim1 dim2 ... dimd`
where dim1, ... , dimd is the d-dimensional representation learned by the graoh embedding method.

In [ ]:
# According to the PK and FK relationships, we build the edgelist
!cat /content/vldb2021_fauce/queries_featurization/graph_embedding/graph/graph.edgelist

1 2
1 3
1 4
1 5
1 6

In [ ]:
!cat /content/vldb2021_fauce/queries_featurization/graph_embedding/emb/graph_node.emd

6 4
1 -0.138841 -0.136080 0.131724 -0.098267
5 0.019718 -0.106719 -0.125513 -0.001253
6 0.036564 -0.028003 0.025221 0.117263
2 0.099814 -0.013354 -0.025868 -0.030946
4 -0.002155 0.120870 0.000698 0.054203
3 -0.109820 0.026476 0.113081 0.022035


## Join Encoding using Joins2Vec

In [ ]:
# !mkdir -p /content/vldb2021_fauce/queries_featurization/example_data/datasets/node_edges
# !touch /content/vldb2021_fauce/queries_featurization/example_data/datasets/node_edges/0.gexf
# !touch /content/vldb2021_fauce/queries_featurization/example_data/datasets/node.Labels

In [11]:
!cat /content/vldb2021_fauce/queries_featurization/Joins2Vec/example_data/datasets/node_edges/0.gexf

<?xml version='1.0' encoding='utf-8'?>
<gexf version="1.1" xmlns="http://www.gexf.net/1.1draft" xmlns:viz="http://www.gexf.net/1.1draft/viz" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.w3.org/2001/XMLSchema-instance">
  <graph defaultedgetype="undirected" mode="static">
    <attributes class="node" mode="static">
      <attribute id="0" title="Label" type="string" />
    </attributes>
    <nodes>
      <node id="1" label="1">
        <attvalues>
          <attvalue for="0" value="1" />
        </attvalues>
      </node>
      <node id="2" label="2">
        <attvalues>
          <attvalue for="0" value="2" />
        </attvalues>
      </node>
      <node id="3" label="3">
        <attvalues>
          <attvalue for="0" value="3" />
        </attvalues>
      </node>
      <node id="4" label="4">
        <attvalues>
          <attvalue for="0" value="4" />
        </attvalues>
      </node>
      <node id="5" label="5">
        <attvalues>
     

Two-node graphs (d=1): 5 (AB, AC, AD, AE, AF)
Three-node graphs (d=2): 10 (ABC, ABD, ABE, ABF, ACD, ACE, ACF, ADE, ADF, AEF)
Four-node graphs (d=3): 10 (ABCD, ABCE, ABCF, ABDE, ABDF, ABEF, ACDE, ACDF, ACEF, ADEF)
Five-node graphs (d=4): 5 (ABCDE, ABCDF, ABCEF, ABDEF, ACDEF)

n=the number of possible
join relationships among the database tables = 5+10+10+5=30

In [34]:
%%bash
source activate qf
cd /content/vldb2021_fauce/queries_featurization/Joins2Vec
# mkdir ../embeddings
python main.py --corpus ./example_data/datasets/node_edges --class_labels_file_name ./example_data/datasets/node.Labels --output_dir ./embeddings -d 30 --wlk_h 2

Device mapping: no known devices.
gradients/nce_loss/embedding_lookup_grad/strided_slice: (StridedSlice): /job:localhost/replica:0/task:0/cpu:0
gradients/nce_loss/Slice_grad/pack: (Pack): /job:localhost/replica:0/task:0/cpu:0
gradients/embedding_lookup_grad/strided_slice: (StridedSlice): /job:localhost/replica:0/task:0/cpu:0
gradients/nce_loss/embedding_lookup_1_grad/strided_slice: (StridedSlice): /job:localhost/replica:0/task:0/cpu:0
gradients/nce_loss/Slice_2_grad/pack: (Pack): /job:localhost/replica:0/task:0/cpu:0
gradients/nce_loss/Slice_1_grad/pack: (Pack): /job:localhost/replica:0/task:0/cpu:0
gradients/nce_loss/Slice_3_grad/pack: (Pack): /job:localhost/replica:0/task:0/cpu:0
gradients/nce_loss/sub_1_grad/Shape_1: (Const): /job:localhost/replica:0/task:0/cpu:0
gradients/nce_loss/truediv_grad/Shape_1: (Const): /job:localhost/replica:0/task:0/cpu:0
gradients/Mean_grad/Prod_1: (Prod): /job:localhost/replica:0/task:0/cpu:0
gradients/Mean_grad/Maximum: (Maximum): /job:localhost/replic

INFO:root:Loaded 1 graph file names form ./example_data/datasets/node_edges
INFO:root:loaded ./example_data/datasets/node_edges/0.gexf a graph with 6 nodes and 5 egdes
INFO:root:converted  as undirected graph
INFO:root:Dumped subgraph2vec sentences for all 1 graphs in ./example_data/datasets/node_edges in 0.0 sec
INFO:root:Initializing SKIPGRAM...
INFO:root:vocabulary size: 19
INFO:root:number of documents: 1
INFO:root:number of words to be trained: 20
I tensorflow/core/common_runtime/direct_session.cc:255] Device mapping:

I tensorflow/core/common_runtime/simple_placer.cc:827] gradients/nce_loss/embedding_lookup_grad/strided_slice: (StridedSlice)/job:localhost/replica:0/task:0/cpu:0
I tensorflow/core/common_runtime/simple_placer.cc:827] gradients/nce_loss/Slice_grad/pack: (Pack)/job:localhost/replica:0/task:0/cpu:0
I tensorflow/core/common_runtime/simple_placer.cc:827] gradients/embedding_lookup_grad/strided_slice: (StridedSlice)/job:localhost/replica:0/task:0/cpu:0
I tensorflow/core/

CalledProcessError: Command 'b'source activate qf\ncd /content/vldb2021_fauce/queries_featurization/Joins2Vec\n# mkdir ../embeddings\npython main.py --corpus ./example_data/datasets/node_edges --class_labels_file_name ./example_data/datasets/node.Labels --output_dir ./embeddings -d 30 --wlk_h 2\n'' returned non-zero exit status 1.

maximum of allowed joins in a query = 4

Two-node graphs (d=1): 5 (AB, AC, AD, AE, AF)
- AB: `B#A`
- AC: `C#A`
- AD: `D#A`
- AE: `E#A`
- AF: `F#A`

Three-node graphs (d=2): 10 (ABC, ABD, ABE, ABF, ACD, ACE, ACF, ADE, ADF, AEF)

Four-node graphs (d=3): 10 (ABCD, ABCE, ABCF, ABDE, ABDF, ABEF, ACDE, ACDF, ACEF, ADEF)

Five-node graphs (d=4): 5 (ABCDE, ABCDF, ABCEF, ABDEF, ACDEF)

n = 5+10+10+5=30

## Columns Encoding

### Build local columns-dependency graphs
- We calculate the Randomized Dependence Coefficient (RDC) values for each pair
of columns in each table $T_i$ from the database $D$. If the RDC value
for two columns exceeds a threshold $𝜏$, then those two columns are
dependent with each other; otherwise, they are independent.
  - The RDC is a measure of nonlinear dependence between two variables.
- we set $𝜏$ as 0.4
- Once there exists a connection
(i.e., an edge) between two columns (i.e., vertices), the graph shows
those two columns are correlated.

Given the global columns-dependency graph, use the same graph embedding method for Table Encoding to transform "Columns" into vectors. Currently, Fauce only focuses on the columns with real-valued numbers.

To build the global columns-dependency graph, it includes 3 steps.
1. For each "Table" in the database, we calculate the Randomized Dependence Coefficient(RDC) values for each pair of "Columns" in each "Table" to build the local columns-dependency graph;
2. Connect the local columns-dependency graph based on the PK and FK relationships among "Tables" to build the global columns-dependency graph;
3. Given the global columns-dependency graph as the input, use the same graph embedding method (the method to represent "Tables" as vectors) to get the embeddings for the "Columns" in the database.

In [ ]:
!cd /content/vldb2021_fauce/queries_featurization/build_graphs && source activate qf && python setup.py install
!cd /content/vldb2021_fauce/queries_featurization/build_graphs && python setup.py install

In [ ]:
!pip install pandas

In [ ]:
%%bash
cd /content/vldb2021_fauce/queries_featurization/build_graphs
python rdc.py --input full_data_12_19_new_CHKK_cluster1.csv --output matrix_12_19_CHKK_cluster1.csv

In [ ]:
%%bash
git clone -b eric  https://github.com/Erostrate9/neurocard.git
chmod +x neurocard/neurocard/scripts/download_imdb.sh
cd neurocard
conda env create -f environment.yml
source activate neurocard
neurocard/scripts/download_imdb.sh

In [ ]:
%%bash
source activate neurocard
python neurocard/neurocard/scripts/prepend_imdb_headers.py

I0816 08:22:35.036232 132448441395008 prepend_imdb_headers.py:76] Prepending header to datasets/job/aka_name.csv: id,person_id,name,imdb_index,name_pcode_cf,name_pcode_nf,surname_pcode,md5sum
I0816 08:22:36.299624 132448441395008 prepend_imdb_headers.py:76] Prepending header to datasets/job/aka_title.csv: id,movie_id,title,imdb_index,kind_id,production_year,phonetic_code,episode_of_id,season_nr,episode_nr,note,md5sum
I0816 08:22:36.820832 132448441395008 prepend_imdb_headers.py:76] Prepending header to datasets/job/cast_info.csv: id,person_id,movie_id,person_role_id,note,nr_order,role_id
I0816 08:22:53.943976 132448441395008 prepend_imdb_headers.py:76] Prepending header to datasets/job/char_name.csv: id,name,imdb_index,imdb_id,name_pcode_nf,surname_pcode,md5sum
I0816 08:22:56.737183 132448441395008 prepend_imdb_headers.py:76] Prepending header to datasets/job/company_name.csv: id,name,country_code,imdb_id,name_pcode_nf,name_pcode_sf,md5sum
I0816 08:22:56.957540 132448441395008 prepend_

In [ ]:
%%bash
cd neurocard
conda env create -f environment.yml
source activate neurocard
python neurocard/scripts/prepend_imdb_headers.py

In [ ]:
!cp -r /content/vldb2021_fauce/queries_featurization/build_graphs/datasets /content/drive/MyDrive/datasets

In [ ]:
%%bash
mkdir /content/vldb2021_fauce/queries_featurization/build_graphs/dataset
cd /content/vldb2021_fauce/queries_featurization/build_graphs/dataset
cp /content/datasets/job/title.csv title.csv
cp /content/datasets/job/cast_info.csv cast_info.csv
cp /content/datasets/job/movie_info.csv movie_info.csv
cp /content/datasets/job/movie_companies.csv movie_companies.csv
cp /content/datasets/job/movie_keyword.csv movie_keyword.csv
cp /content/datasets/job/movie_info_idx.csv movie_info_idx.csv

In [ ]:
import pandas as pd
df_title = pd.read_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/title.csv", escapechar='\\')
# df_title = df_title[['id', 'kind_id', 'production_year']]
# df_title = df_title.dropna()
# df_title.to_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/title.csv", index=False)
df_title

,id,kind_id,production_year
0,80889,7,1980.0
1,5156,7,2010.0
2,197772,7,1962.0
3,111913,7,2012.0
4,40704,7,1971.0
...,...,...,...
2456212,2528270,1,1953.0
2456213,2528296,2,1985.0
2456214,2528298,3,2012.0
2456215,2528268,1,1942.0


In [ ]:
df_ci = pd.read_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/cast_info.csv", escapechar='\\')
df_ci
# df_ci = df_ci[['movie_id', 'role_id']]
# df_ci = df_ci.dropna()
# df_ci.to_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/cast_info.csv", index=False)

,movie_id,role_id
0,968504,1
1,2163857,1
2,2324528,1
3,1851347,1
4,1681365,1
...,...,...
36244339,1762387,11
36244340,2524799,11
36244341,1966201,11
36244342,2479370,11


In [ ]:
df_mi = pd.read_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/movie_info.csv", escapechar='\\')
df_mi
# df_mi = df_mi[['movie_id', 'info_type_id']]
# df_mi = df_mi.dropna()
# df_mi.to_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/movie_info.csv", index=False)

,movie_id,info_type_id
0,921521,4
1,921522,4
2,921523,4
3,921524,4
4,921525,4
...,...,...
14835715,921516,4
14835716,921517,4
14835717,921518,4
14835718,921519,4


In [ ]:
df_mc = pd.read_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/movie_companies.csv", escapechar='\\')
df_mc
# df_mc = df_mc[['company_id', 'company_type_id', 'movie_id']]
# df_mc = df_mc.dropna()
# df_mc.to_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/movie_companies.csv", index=False)

,company_id,company_type_id,movie_id
0,1,1,2
1,1,1,2
2,2,1,11
3,3,1,44
4,4,1,50
...,...,...,...
2609124,28437,2,2525599
2609125,140876,2,2525600
2609126,234997,2,2525601
2609127,168049,2,2525602


In [ ]:
df_mk = pd.read_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/movie_keyword.csv", escapechar='\\')
# df_mk = df_mk[['movie_id', 'keyword_id']]
# df_mk = df_mk.dropna()
# df_mk.to_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/movie_keyword.csv", index=False)
df_mk

,movie_id,keyword_id
0,2,1
1,11,2
2,22,2
3,44,3
4,24,2
...,...,...
4523925,2525602,8621
4523926,2525602,5761
4523927,2525602,24715
4523928,2525602,8386


In [ ]:
df_mi_idx = pd.read_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/movie_info_idx.csv", escapechar='\\')
# df_mi_idx = df_mi_idx[['info_type_id', 'movie_id']]
# df_mi_idx = df_mi_idx.dropna()
# df_mi_idx.to_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/movie_info_idx.csv", index=False)
df_mi_idx

,info_type_id,movie_id
0,99,2
1,100,2
2,101,2
3,99,11
4,100,11
...,...,...
1380030,113,2207311
1380031,113,1730276
1380032,113,2378686
1380033,113,2322570


In [ ]:
%%bash
cd /content/vldb2021_fauce/queries_featurization/build_graphs
python rdc.py --input dataset/title.csv --output output_matrix/matrix_title.csv
python rdc.py --input dataset/cast_info.csv --output output_matrix/matrix_cast_info.csv
python rdc.py --input dataset/movie_info.csv --output output_matrix/movie_info.csv
python rdc.py --input dataset/movie_companies.csv --output output_matrix/movie_companies.csv
python rdc.py --input dataset/movie_keyword.csv --output output_matrix/movie_keyword.csv
python rdc.py --input dataset/movie_info_idx.csv --output output_matrix/movie_info_idx.csv

Process is terminated.


In [ ]:
%%bash
cd /content/vldb2021_fauce/queries_featurization/build_graphs
python rdc.py --input dataset/title.csv --output output_matrix/matrix_title.csv

In [ ]:
%%bash
cd /content/vldb2021_fauce/queries_featurization/build_graphs
python rdc.py --input dataset/cast_info.csv --output output_matrix/matrix_cast_info.csv

In [ ]:
%%bash
cd /content/vldb2021_fauce/queries_featurization/build_graphs
python rdc.py --input dataset/movie_info.csv --output output_matrix/movie_info.csv

In [ ]:
%%bash
cd /content/vldb2021_fauce/queries_featurization/build_graphs
python rdc.py --input dataset/movie_companies.csv --output output_matrix/movie_companies.csv

In [ ]:
%%bash
cd /content/vldb2021_fauce/queries_featurization/build_graphs
python rdc.py --input dataset/movie_keyword.csv --output output_matrix/movie_keyword.csv

In [ ]:
%%bash
cd /content/vldb2021_fauce/queries_featurization/build_graphs
python rdc.py --input dataset/movie_info_idx.csv --output output_matrix/movie_info_idx.csv

In [ ]:
"""
http://papers.nips.cc/paper/5138-the-randomized-dependence-coefficient.pdf
"""
import argparse
import numpy as np
from scipy.stats import rankdata
import csv
import pandas

def rdc(x, y, f=np.sin, k=20, s=1/6., n=1):
    """
    Computes the Randomized Dependence Coefficient
    x,y: numpy arrays 1-D or 2-D
         If 1-D, size (samples,)
         If 2-D, size (samples, variables)
    f:   function to use for random projection
    k:   number of random projections to use
    s:   scale parameter
    n:   number of times to compute the RDC and
         return the median (for stability)

    According to the paper, the coefficient should be relatively insensitive to
    the settings of the f, k, and s parameters.
    """
    if n > 1:
        values = []
        for i in range(n):
            try:
                values.append(rdc(x, y, f, k, s, 1))
            except np.linalg.linalg.LinAlgError: pass
        return np.median(values)

    if len(x.shape) == 1: x = x.reshape((-1, 1))
    if len(y.shape) == 1: y = y.reshape((-1, 1))

    # Copula Transformation
    cx = np.column_stack([rankdata(xc, method='ordinal') for xc in x.T])/float(x.size)
    cy = np.column_stack([rankdata(yc, method='ordinal') for yc in y.T])/float(y.size)

    # Add a vector of ones so that w.x + b is just a dot product
    O = np.ones(cx.shape[0])
    X = np.column_stack([cx, O])
    Y = np.column_stack([cy, O])

    # Random linear projections
    Rx = (s/X.shape[1])*np.random.randn(X.shape[1], k)
    Ry = (s/Y.shape[1])*np.random.randn(Y.shape[1], k)
    X = np.dot(X, Rx)
    Y = np.dot(Y, Ry)

    # Apply non-linear function to random projections
    fX = f(X)
    fY = f(Y)

    # Compute full covariance matrix
    C = np.cov(np.hstack([fX, fY]).T)

    # Due to numerical issues, if k is too large,
    # then rank(fX) < k or rank(fY) < k, so we need
    # to find the largest k such that the eigenvalues
    # (canonical correlations) are real-valued
    k0 = k
    lb = 1
    ub = k
    while True:

        # Compute canonical correlations
        Cxx = C[:k, :k]
        Cyy = C[k0:k0+k, k0:k0+k]
        Cxy = C[:k, k0:k0+k]
        Cyx = C[k0:k0+k, :k]

        eigs = np.linalg.eigvals(np.dot(np.dot(np.linalg.pinv(Cxx), Cxy),
                                        np.dot(np.linalg.pinv(Cyy), Cyx)))

        # Binary search if k is too large
        if not (np.all(np.isreal(eigs)) and
                0 <= np.min(eigs) and
                np.max(eigs) <= 1):
            ub -= 1
            k = (ub + lb) // 2
            continue
        if lb == ub: break
        lb = k
        if ub == lb + 1:
            k = ub
        else:
            k = (ub + lb) // 2

    return np.sqrt(np.max(eigs))

In [ ]:
# import sys
# directory_path = '/content/vldb2021_fauce/queries_featurization/build_graphs'
# sys.path.append(directory_path)
# from rdc import rdc
id=df_title['id'].to_numpy()
kind_id=df_title['kind_id'].to_numpy()
production_year=df_title['production_year'].to_numpy()
print(rdc(id, id))
print(rdc(id, kind_id))
print(rdc(id, production_year))
print(rdc(kind_id, id))
print(rdc(kind_id, kind_id))
print(rdc(kind_id, production_year))
print(rdc(production_year, id))
print(rdc(production_year, kind_id))
print(rdc(production_year, production_year))

0.9999999774431265
0.8986855254234885
0.29937012967382365
0.8950939051139094
0.9999998493808534
0.3584660073334791
0.2992931749678756
0.3586904011702757
0.9999999998828576


In [ ]:
# df_title = df_title[['id', 'kind_id', 'production_year']]
!cat /content/vldb2021_fauce/queries_featurization/build_graphs/output_matrix/matrix_title.csv
# title correlated columns: title.id-title.kind_id
# Local dependency graph:
# vertex: id, kind_id, production_year
# edge: id - kind_id

0.9999999991558958,0.8948306112007808,0.29930571484881907
0.8947442076828466,0.9999999935391886,0.3584717342951818
0.29929748222719493,0.3584879992153889,0.9999999823392659


In [ ]:
# df_ci = df_ci[['movie_id', 'role_id']]
!cat /content/vldb2021_fauce/queries_featurization/build_graphs/output_matrix/matrix_cast_info.csv
# cast_info correlated columns: none
# Local dependency graph:
# vertex: movie_id, role_id
# edge: None

0.9999997058739752,0.03099708436444811
0.04700204139704528,0.9999999313443979


In [ ]:
# df_mi = df_mi[['movie_id', 'info_type_id']]
!cat /content/vldb2021_fauce/queries_featurization/build_graphs/output_matrix/matrix_movie_info.csv
# movie_info correlated columns: none
# Local dependency graph:
# vertex: movie_id, info_type_id
# edge: None

0.9997449862941917,0.2912571120561066
0.2797470378981132,0.9999994955628929


In [ ]:
# df_mc = df_mc[['company_type_id', 'movie_id']]
!cat /content/vldb2021_fauce/queries_featurization/build_graphs/output_matrix/matrix_movie_companies.csv
# movie_companies.company_type_id-movie_companies.movie_id

# Local dependency graph:
# vertex: company_type_id, movie_id
# edge: company_type_id - movie_id

0.9999999932252222,0.6099719665365848,0.4075593546429689
0.6583993504680015,0.9999999999766027,0.45466136180660693
0.4078225372873896,0.7615193066518248,0.9999989692243756


In [ ]:
# df_mk = df_mk[['movie_id', 'keyword_id']]
!cat /content/vldb2021_fauce/queries_featurization/build_graphs/output_matrix/matrix_movie_keyword.csv
# movie_keyword correlated columns: none

# Local dependency graph:
# vertex: movie_id, keyword_id
# edge: none

0.9999999999976192,0.07101817748633753
0.07101353654628673,0.9999999917224112


In [ ]:
# df_mi_idx = df_mi_idx[['info_type_id', 'movie_id']]
!cat /content/vldb2021_fauce/queries_featurization/build_graphs/output_matrix/matrix_movie_info_idx.csv
# movie_info_idx correlated columns: movie_info_idx.info_type_id-movie_info_idx.movie_id

# Local dependency graph:
# vertex: info_type_id, movie_id
# edge: info_type_id - movie_id

0.9999999994780473,0.5624087905242087
0.5625584262602452,0.9999999985313216


- For each "Table" in the database, we calculate the Randomized Dependence Coefficient(RDC) values for each pair of "Columns" in each "Table" to build the local columns-dependency graph;
  - title: title.id-title.kind_id
  - cast_info: none
  - movie_info: none
  - movie_companies: movie_companies.company_type_id-movie_companies.movie_id
  - movie_keyword: none
  - movie_info_idx: movie_info_idx.info_type_id-movie_info_idx.movie_id
```csv
title.id movie_id 1
kind_id 2
production_year 3
role_id 4
info_type_id 5
company_type_id 6
keyword_id 7
```

### Build a global columns-dependency graph for the database
Connect the local columns-dependency graph based on the PK and FK relationships among "Tables" to build the global columns-dependency graph;

In [95]:
!cat /content/vldb2021_fauce/queries_featurization/graph_embedding/graph/global_column_names.txt

title.id 1
title.kind_id 2
title.production_year 3
cast_info.movie_id 1
cast_info.role_id 4
movie_info.movie_id 1
movie_info.info_type_id 5
movie_companies.company_type_id 6
movie_companies.movie_id 1
movie_keyword.movie_id 1
movie_keyword.keyword_id 7
movie_info_idx.info_type_id 5
movie_info_idx.movie_id 1

local columns-dependency graph;
  - title: title.id-title.kind_id
  - cast_info: none
  - movie_info: none
  - movie_companies: movie_companies.company_id-movie_companies.company_type_id, movie_companies.company_id-movie_companies.movie_id,  movie_companies.company_type_id-movie_companies.movie_id
  - movie_keyword: none
  - movie_info_idx: movie_info_idx.info_type_id-movie_info_idx.movie_id
```csv
title.id movie_id 1
kind_id 2
production_year 3
role_id 4
info_type_id 5
company_type_id 6
keyword_id 7
```

In [100]:
!cat /content/vldb2021_fauce/queries_featurization/graph_embedding/graph/global_columns_dependency_graph.edgelist

1 2
6 1
1 5
3 3
4 4
7 7

### Use graph embedding for encoding
Given the global columns-dependency graph as the input, use the same graph embedding method (the method to represent "Tables" as vectors) to get the embeddings for the "Columns" in the database.

In [99]:
# C = 7
%%bash
cd /content/vldb2021_fauce/queries_featurization/graph_embedding
source activate qf
python main.py --input graph/global_columns_dependency_graph.edgelist --output emb/global_columns.emd --dimensions 7

Walk iteration:
1 / 3
2 / 3
3 / 3


/usr/local/envs/qf/lib/python2.7/site-packages/gensim/models/word2vec.py:651: UserWarning: C extension not loaded for Word2Vec, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn("C extension not loaded for Word2Vec, training will be slow. "


## Combine all vectors
| Type    | Table       | Join          | Column          | Predicate   |
|---------|-------------|---------------|-----------------|-------------|
| Segment | ⟨Tables⟩    | ⟨Joins⟩       | ⟨Columns⟩       | ⟨Values⟩    |
| Method  | Embedding   | Joins2Vec     | Columns2Vec     | Range       |
| Seg. Size | m⌈log(m+1)⌉ | n           | C               | 2 × C       |

- m: the number of tables in the database
  - JOB-light: 6
- n: the number of possible join relationships among the database tables
  - JOB-light: 30
- C: the total number of different columns in the database
  - JOB-light: 7
- the feature vector has a length of $L = m\lceil log(m+1)\rceil+n+3C=6*3+30+3*7=69$

In [7]:
import pandas as pd
df = pd.read_csv("/content/vldb2021_fauce/training_data/job-light-content-2500.csv", sep='#', header=None, names=['Tables', 'Joins', 'Filters', 'Cardinality'])
df

,Tables,Joins,Filters,Cardinality
0,"movie_companies,movie_info_idx,title","title.id=movie_companies.movie_id,movie_compan...","movie_info_idx.info_type_id,=,101.0,title.kind...",66264
1,"movie_companies,movie_info_idx,title","title.id=movie_companies.movie_id,movie_compan...","movie_info_idx.info_type_id,=,101.0,title.prod...",64064
2,"movie_companies,movie_info_idx,title","title.id=movie_companies.movie_id,movie_compan...","title.kind_id,=,4.0,title.production_year,=,19...",1161
3,"movie_companies,movie_info_idx,title","title.id=movie_companies.movie_id,movie_compan...","title.kind_id,=,1.0,movie_info_idx.info_type_i...",302610
4,"movie_companies,movie_info_idx,title","title.id=movie_companies.movie_id,movie_compan...","title.kind_id,=,7.0,title.production_year,<=,2...",287550
...,...,...,...,...
2203,"cast_info,movie_companies,movie_info,movie_key...","title.id=movie_info.movie_id,movie_info.movie_...","title.kind_id,=,1.0,cast_info.role_id,=,7.0,ti...",225
2204,"cast_info,movie_companies,movie_info,movie_key...","title.id=movie_info.movie_id,movie_info.movie_...","movie_keyword.keyword_id,=,9629.0,title.kind_i...",1953488
2205,"cast_info,movie_companies,movie_info,movie_key...","title.id=movie_info.movie_id,movie_info.movie_...","title.production_year,>=,2010.0,cast_info.role...",4209673905
2206,"cast_info,movie_companies,movie_info,movie_key...","title.id=movie_info.movie_id,movie_info.movie_...","title.kind_id,=,1.0,cast_info.role_id,=,1.0,mo...",70983


In [12]:
df['n_joins'] = df['Joins'].apply(lambda x: len(x.split(',')))
min_n_joins = df['n_joins'].min()
max_n_joins = df['n_joins'].max()

print(f"min(n_joins): {min_n_joins}")
print(f"max(n_joins): {max_n_joins}")

min(n_joins): 1
max(n_joins): 4


### Tables

```csv
title 1
cast_info 2
movie_info 3
movie_companies 4
movie_keyword 5
movie_info_idx 6
```

In [6]:
import numpy as np
def vectorize_tables(tables, f_emb='/content/vldb2021_fauce/queries_featurization/graph_embedding/emb/graph_node.emd',
                     f_table_names='/content/vldb2021_fauce/queries_featurization/graph_embedding/graph/table_names.txt'):
  """convert tables into a single vector
    Args:
      tables: string, Comma-separated table names
      f_emb: generated embedding file, its first line is "<number of tables> <number of embedding size>", e.g.: 6 3
        other lines are: <table_id> <embedding_feature_1> <embedding_feature_2> <embedding_feature_3> ...
      f_table_names: consists of <number of tables> lines, each line is: <table_name> <table_id>  e.g.: title 1

    Returns:
      Vectorization of input tables, its feature order is sorted by <table_id>. Numpy arry, Vector size is (1, <number of tables> * <number of embedding size>)
      All tables that are not included in tables, the vector is filled with 0.
    E.g.: input: tables = "cast_info,title",
                content in f_emb = "
                            3 2
                            2 0.12 0.1
                            1 0.11 0.3
                            3 0.5 0.7
                          "
                content in f_table_names =
                "
                    title 1
                    cast_info 2
                    movie_info 3
                "
          output: [0.11, 0.3, 0.12, 0.1, 0, 0]
    """
    # Read embedding file
  with open(f_emb, 'r') as f:
    num_tables, embedding_size = map(int, f.readline().strip().split())
    embeddings = {}
    for line in f:
        parts = line.strip().split()
        table_id = int(parts[0])
        embedding = list(map(float, parts[1:]))
        embeddings[table_id] = embedding

    # Read table names file
  with open(f_table_names, 'r') as f:
    table_id_map = {}
    for line in f:
        table_name, table_id = line.strip().split()
        table_id_map[table_name] = int(table_id)
    # Process input tables
  input_tables = tables.split(',')
  # Create the output vector
  output_vector = np.zeros((1, num_tables * embedding_size))

  # Fill the output vector with embeddings for input tables
  for table in input_tables:
      if table in table_id_map:
          table_id = table_id_map[table]
          if table_id in embeddings:
              start_idx = (table_id - 1) * embedding_size
              output_vector[0, start_idx:start_idx+embedding_size] = embeddings[table_id]

  return output_vector

In [10]:
res = vectorize_tables("movie_companies,movie_info_idx,title")
print(res)
print(res.shape)


[[-0.18052  -0.171498  0.166586  0.        0.        0.        0.
   0.        0.       -0.002204  0.161061  0.000822  0.        0.
   0.        0.049148 -0.038398  0.034771]]
(1, 18)


### Joins

In [43]:
import numpy as np
def vectorize_joins(joins, f_emb='/content/vldb2021_fauce/queries_featurization/Joins2Vec/embeddings/node_edges_dims_30_epochs_3_embeddings.txt',
                     f_table_names='/content/vldb2021_fauce/queries_featurization/graph_embedding/graph/table_names.txt'):
  """convert tables into a single vector
    Args:
      joins: string, Comma-separated join names, e.g., "title.id=cast_info.movie_id,title.id=movie_companies.movie_id"
      f_emb: generated embedding file, its first line is "<number_of_join_relationships> <number_of_embedding_size>", e.g.: 19 30
        other lines are: <join_relationship> <embedding_feature_1> <embedding_feature_2> <embedding_feature_3> ...
        <join_relationship> is in form of <table_id>#<title_id>#...#<table_id>.
        We only consider 1-degree join, e.g., 1#2
      f_table_names: consists of <number of tables> lines, each line is: <table_name> <table_id>  e.g.: title 1

    Returns:
      Vectorization of input joins, summing up embeddings of existing join relationships in Joins. Numpy arry, Vector size is (1, <number of embedding size>)

    E.g.: input: joins = "title.id=movie_info.movie_id,title.id=cast_info.movie_id,cast_info.movie_id=movie_info.movie_id",
                content in f_emb = "
                            3 2
                            2#1 0.12 0.1
                            3#1 0.11 0.3
                            2#1##1#3 0.5 0.7
                            UNK -1.0 -1.0
                          "
                content in f_table_names =
                "
                    title 1
                    cast_info 2
                    movie_info 3
                "
          joins can be converted into: "1#3,1#2,2#3", which is equivalent to "3#1,2#1,UNK", because 2#3 does not exist in f_emb
          so we need to sum up the embedding of 3#1, 2#1 and UNK i.e., [0.11 0.3] + [0.12 0.1] +[-1.0,-1.0].
          output: [-0.77,-0.6]
    """
  # Read embedding file
  with open(f_emb, 'r') as f:
      num_join_relationships, embedding_size = map(int, f.readline().strip().split())
      embeddings = {}
      for line in f:
          parts = line.strip().split()
          join_relationship = parts[0]
          embedding = np.array(list(map(float, parts[1:])))
          embeddings[join_relationship] = embedding
  # Read table names file
  with open(f_table_names, 'r') as f:
      table_id_map = {line.strip().split()[0]: int(line.strip().split()[1]) for line in f}
      id_table_map = {v: k for k, v in table_id_map.items()}

  # Process input joins
  input_joins = joins.split(',')

  # Convert join names to table IDs
  converted_joins = []
  for join in input_joins:
      table1, table2 = join.split('=')
      table1_name, _ = table1.split('.')
      table2_name, _ = table2.split('.')
      id1, id2 = table_id_map[table1_name], table_id_map[table2_name]
      converted_joins.append(f"{id1}#{id2}")
  # Sum up the embeddings
  output_vector = np.zeros(embedding_size)
  for join in converted_joins:
      if join in embeddings:
          output_vector += embeddings[join]
      # Check for reverse join (e.g., "2#1" instead of "1#2")
      elif '#'.join(reversed(join.split('#'))) in embeddings:
          output_vector += embeddings['#'.join(reversed(join.split('#')))]
      else:
          # Handle unknown join relationships
          output_vector += embeddings['UNK']
  return output_vector.reshape(1, -1)

In [44]:
joins = "title.id=movie_companies.movie_id,movie_companies.movie_id=movie_info_idx.movie_id"
res = vectorize_joins(joins)
print(res)
print(res.shape)

[[ 1.33223236e-01  7.15297133e-01 -2.29930609e-01 -2.19643116e-04
  -2.00131482e+00  1.27593519e+00  1.69880942e+00 -1.23531461e-01
   1.25661343e+00 -6.59069829e-01 -1.44282746e+00  1.28127548e+00
  -6.00862056e-01  2.77778250e+00 -2.62781203e-01  2.29831254e+00
  -1.53330207e+00 -2.36716813e+00  1.04958715e+00 -2.95567536e+00
  -6.61737800e-01  8.85435194e-02 -2.81342804e-01  4.92805243e-02
  -1.91448063e+00 -2.75257182e+00 -1.56192169e+00  2.24238753e-01
   2.27742860e+00 -5.81855237e-01]]
(1, 30)


In [50]:
!cp "/content/drive/MyDrive/Colab Notebooks/Fauce.ipynb" /content/vldb2021_fauce/colab_notebooks/Fauce.ipynb

cp: error reading '/content/drive/MyDrive/Colab Notebooks/Fauce.ipynb': No such file or directory


### Columns

### Values